# CAPSTONE Final Project 

## A Martial Arts Teacher wants to set up his office in San Francisco . He wants to set his office in the location where the crime rates are high(as people are more willing to learn martial arts) and also in the locality where people Visit more often like places near restaurants,gyms etc. 
## So In order to solve this problem we will be taking the following steps:
 ### 1.Taking the data base from police and checking which area has high crime Rates.
 ### 2.Then using our fourSquare API we will determine the best neighbor hood. 



In [76]:
#importing Libraries
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!pip install geopy # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print("Libraries Imported!")

Solving environment: done

# All requested packages already installed.

Libraries Imported!


In [77]:
df_area = pd.read_csv('https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DV0101EN/labs/Data_Files/Police_Department_Incidents_-_Previous_Year__2016_.csv')

print('Dataset downloaded and read into a pandas dataframe!')

Dataset downloaded and read into a pandas dataframe!


In [78]:
df_area.head()

,IncidntNum,Category,Descript,DayOfWeek,Date,Time,PdDistrict,Resolution,Address,X,Y,Location,PdId
0,120058272,WEAPON LAWS,POSS OF PROHIBITED WEAPON,Friday,01/29/2016 12:00:00 AM,11:00,SOUTHERN,"ARREST, BOOKED",800 Block of BRYANT ST,-122.403405,37.775421,"(37.775420706711, -122.403404791479)",12005827212120
1,120058272,WEAPON LAWS,"FIREARM, LOADED, IN VEHICLE, POSSESSION OR USE",Friday,01/29/2016 12:00:00 AM,11:00,SOUTHERN,"ARREST, BOOKED",800 Block of BRYANT ST,-122.403405,37.775421,"(37.775420706711, -122.403404791479)",12005827212168
2,141059263,WARRANTS,WARRANT ARREST,Monday,04/25/2016 12:00:00 AM,14:59,BAYVIEW,"ARREST, BOOKED",KEITH ST / SHAFTER AV,-122.388856,37.729981,"(37.7299809672996, -122.388856204292)",14105926363010
3,160013662,NON-CRIMINAL,LOST PROPERTY,Tuesday,01/05/2016 12:00:00 AM,23:50,TENDERLOIN,NONE,JONES ST / OFARRELL ST,-122.412971,37.785788,"(37.7857883766888, -122.412970537591)",16001366271000
4,160002740,NON-CRIMINAL,LOST PROPERTY,Friday,01/01/2016 12:00:00 AM,00:30,MISSION,NONE,16TH ST / MISSION ST,-122.419672,37.765050,"(37.7650501214668, -122.419671780296)",16000274071000


In [79]:
df_area.rename(columns={'X':'Longitude'},inplace=True)
df_area.rename(columns={'Y':'Latitude'},inplace=True)

In [80]:
df_area.head()

,IncidntNum,Category,Descript,DayOfWeek,Date,Time,PdDistrict,Resolution,Address,Longitude,Latitude,Location,PdId
0,120058272,WEAPON LAWS,POSS OF PROHIBITED WEAPON,Friday,01/29/2016 12:00:00 AM,11:00,SOUTHERN,"ARREST, BOOKED",800 Block of BRYANT ST,-122.403405,37.775421,"(37.775420706711, -122.403404791479)",12005827212120
1,120058272,WEAPON LAWS,"FIREARM, LOADED, IN VEHICLE, POSSESSION OR USE",Friday,01/29/2016 12:00:00 AM,11:00,SOUTHERN,"ARREST, BOOKED",800 Block of BRYANT ST,-122.403405,37.775421,"(37.775420706711, -122.403404791479)",12005827212168
2,141059263,WARRANTS,WARRANT ARREST,Monday,04/25/2016 12:00:00 AM,14:59,BAYVIEW,"ARREST, BOOKED",KEITH ST / SHAFTER AV,-122.388856,37.729981,"(37.7299809672996, -122.388856204292)",14105926363010
3,160013662,NON-CRIMINAL,LOST PROPERTY,Tuesday,01/05/2016 12:00:00 AM,23:50,TENDERLOIN,NONE,JONES ST / OFARRELL ST,-122.412971,37.785788,"(37.7857883766888, -122.412970537591)",16001366271000
4,160002740,NON-CRIMINAL,LOST PROPERTY,Friday,01/01/2016 12:00:00 AM,00:30,MISSION,NONE,16TH ST / MISSION ST,-122.419672,37.765050,"(37.7650501214668, -122.419671780296)",16000274071000


In [81]:
df_area.shape

(150500, 13)

In [82]:
#Let's take first 100 values
limit=100
df_area=df_area.iloc[0:limit, :]

In [83]:
df_area.shape

(100, 13)

In [84]:
# finding the latitude and longitude of San Francisco
address= 'San Francisco , California'
geolocator=Nominatim(user_agent='area_explorer')
location=geolocator.geocode(address)
latitude=location.latitude
longitude=location.longitude
print(latitude,longitude)


37.7790262 -122.4199061


In [85]:
sanfran_map= folium.Map(location=[latitude,longitude],zoom_start=12)
sanfran_map

## Now we will place the data onto the map

In [86]:
incidents = folium.map.FeatureGroup()

# loop through the 100 crimes and add each to the incidents feature group
for lat, lng, in zip(df_area.Latitude, df_area.Longitude):
    incidents.add_child(
        folium.features.CircleMarker(
            [lat, lng],
            radius=5, # define how big you want the circle markers to be
            color='green',
            fill=True,
            fill_color='red',
            fill_opacity=0.6
        )
    )

# add incidents to map
sanfran_map.add_child(incidents)

In [87]:
# now we will display data in from of clusters so to get clear picture of the data 
from folium import plugins

# let's start again with a clean copy of the map of San Francisco
sanfran_map = folium.Map(location = [latitude, longitude], zoom_start = 12)

# instantiate a mark cluster object for the incidents in the dataframe
incidents = plugins.MarkerCluster().add_to(sanfran_map)

# loop through the dataframe and add each data point to the mark cluster
for lat, lng, label, in zip(df_area.Latitude, df_area.Longitude, df_area.Category):
    folium.Marker(
        location=[lat, lng],
        icon=None,
        popup=label,
    ).add_to(incidents)


sanfran_map

## Now we can clearly see the data on the map ,which area has high crime rate and which area has low crime rate 

In [88]:
df_area.groupby('Address')['PdDistrict'].count().sort_values(ascending=False)[:10]

Address
800 Block of BRYANT ST     3
3RD ST / EGBERT AV         3
23RD ST / FOLSOM ST        2
HAYES ST / FRANKLIN ST     2
1500 Block of BRYANT ST    2
900 Block of MARKET ST     2
9TH ST / MISSION ST        2
BRYANT ST / ZOE ST         2
18TH ST / GUERRERO ST      2
1000 Block of COLE ST      2
Name: PdDistrict, dtype: int64

#### Now this is a very important observation as now have the data from which we can determine crime areas

In [89]:
#we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.
CLIENT_ID = 'SFKLP2QNSKQ4ZRTJN2TWN35H500O45ORSBVIZ1URMZLX4FBO' # your Foursquare ID
CLIENT_SECRET = 'Q2DEUR5VU50HB2BHGYMQDE5B0AQ2TAJCT0QJWD1D2IK4LVHL' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: SFKLP2QNSKQ4ZRTJN2TWN35H500O45ORSBVIZ1URMZLX4FBO
CLIENT_SECRET:Q2DEUR5VU50HB2BHGYMQDE5B0AQ2TAJCT0QJWD1D2IK4LVHL


In [90]:
# now we dont need that much data from the table
df_new=df_area[['PdDistrict','Address','Latitude','Longitude']]
df_new.groupby(['PdDistrict'])['Address','Latitude','Longitude'].apply(lambda x:', '.join(x)).sort_values(ascending=False).to_frame().reset_index()
df_new.head()


/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  This is separate from the ipykernel package so we can avoid doing imports until


,PdDistrict,Address,Latitude,Longitude
0,SOUTHERN,800 Block of BRYANT ST,37.775421,-122.403405
1,SOUTHERN,800 Block of BRYANT ST,37.775421,-122.403405
2,BAYVIEW,KEITH ST / SHAFTER AV,37.729981,-122.388856
3,TENDERLOIN,JONES ST / OFARRELL ST,37.785788,-122.412971
4,MISSION,16TH ST / MISSION ST,37.765050,-122.419672


## Now We have a table according to which we can find the neighborhood data

### Now using API to find near by venues

In [91]:
Latitude_1=df_new.loc[0,'Latitude']
Longitude_1=df_new.loc[0,'Longitude']
near_by_place=df_new.loc[0,'Address']
print('Latitude and longitude values of {} are {}, {}.'.format(near_by_place, 
                                                               Latitude_1, 
                                                               Longitude_1))

Latitude and longitude values of 800 Block of BRYANT ST are 37.775420706711, -122.40340479147899.


In [92]:
# lets check for venues which are under 500 meter radius
LIMIT=15
radius=50
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    Latitude_1, 
    Longitude_1, 
    radius, 
    LIMIT)
url 

'https://api.foursquare.com/v2/venues/explore?&client_id=SFKLP2QNSKQ4ZRTJN2TWN35H500O45ORSBVIZ1URMZLX4FBO&client_secret=Q2DEUR5VU50HB2BHGYMQDE5B0AQ2TAJCT0QJWD1D2IK4LVHL&v=20180605&ll=37.775420706711,-122.40340479147899&radius=50&limit=15'

In [93]:
results=requests.get(url).json()

In [94]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']


In [95]:
#sorting data
venues=results['response']['groups'][0]['items']
nearby_venues=json_normalize(venues)

filtered_columns=['venue.name','venue.categories','venue.location.lat','venue.location.lng']
nearby_venues=nearby_venues.loc[:, filtered_columns]

nearby_venues['venue.categories']=nearby_venues.apply(get_category_type , axis=1)
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,Dinosaurs,Vietnamese Restaurant,37.775013,-122.403349
1,T-Dub's on Fridays,BBQ Joint,37.775150,-122.403394
2,Golden Burma,Burmese Restaurant,37.775184,-122.403455
3,Ted's Sports Bar & Grill,Sports Bar,37.775407,-122.402907
4,Cafe In,Café,37.775071,-122.403591


In [96]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

5 venues were returned by Foursquare.


In [97]:
nearby_venues

,name,categories,lat,lng
0,Dinosaurs,Vietnamese Restaurant,37.775013,-122.403349
1,T-Dub's on Fridays,BBQ Joint,37.775150,-122.403394
2,Golden Burma,Burmese Restaurant,37.775184,-122.403455
3,Ted's Sports Bar & Grill,Sports Bar,37.775407,-122.402907
4,Cafe In,Café,37.775071,-122.403591


## Now super imposing the venues in the map to get clear picture and our client can clearly visualize the data.  

In [100]:
sanfran_new_map=folium.Map(location=[37.78,-122.42],zoom_start=15)    # 37.7790262 -122.4199061
sanfran_new_map

In [102]:
incidents_1 = folium.map.FeatureGroup()


for lat_1, lng_1, in zip(nearby_venues.lat, nearby_venues.lng):
    incidents_1.add_child(
        folium.features.CircleMarker(
            [lat_1, lng_1],
            radius=8, # define how big you want the circle markers to be
            color='red',
            fill=True,
            fill_color='yellow',
            fill_opacity=0.6
        )
    )

# add incidents to map
sanfran_new_map.add_child(incidents_1)

# We have shown  our customer the most appropriate place with near by venues
